In [1]:
import numpy as np
#import mdwc md_suit 
import mdwc.MD_suite.MD_suite as md_ft
#import mdwc md parameters reader
import mdwc.software_tools.abinit_controller as ac
#import ase calculators for energies and forces
import ase
from ase.calculators.emt import EMT
from ase.lattice.cubic import FaceCenteredCubic
from ase.calculators.lj import LennardJones


In [2]:
#start the sigma from stress from ase
def get_sigma(stress):
#stress= atoms.get_stress()# pressumable in eV/A^3
    sigma= np.array([[stress[0],stress[5],stress[4]], [stress[5], stress[1],stress[3]],\
                     [stress[4],stress[3],stress[2]]])
    return sigma

In [3]:
#starting ase potential
#3.446 angstrom
#8.5 angstrom
# 0.010768 eV
LJ= LennardJones(epsilon= 0.010768, sigma= 3.446, rc= 8.5)

#start structure from ase
atoms = FaceCenteredCubic(size=(3,3,3), symbol="Ar", pbc=True)
atoms.set_calculator(LJ)

In [4]:
# import parameters for the md, from the jn_test.md file 
Qmass, bmass, P_ext, dt, correc_steps, md_steps, abinit_steps= ac.get_md_parameters('jn_test.md')

numb_bond_cons, bool_bond_cons, numb_angl_cons, bool_angl_cons, \
numb_cell_para_cons, bool_cell_para_cons, numb_cell_angl_cons, bool_cell_angl_cons,\
volu_cons , bool_volu, numb_atom_fix_cons, bool_atom_fix_cons, bond_const, angl_const, cell_para_const,\
cell_angl_const, atom_fix_const, bond_valu, angl_valu, cell_para_valu,\
cell_angl_valu, volu_valu, atom_fix_valu, atom_fix_cord= ac.get_md_constrains('jn_test.md')

md_total_steps= md_steps*abinit_steps

temp_arra= ac.temp_data_reader('jn_test.md', md_total_steps)

#paremeters to start the thermostat degree of freedom (don't touch)
s_t=1.0 #thermostat degree of freedom
s_t_dot= 0.0#time derivative of thermostat degree of freedom



In [5]:
#this part is equivalent to ac.get_nat_mass_latvec_in_strten_in
#in the mdwc_ script
mass=   atoms.get_masses()#amu
h_t= atoms.cell#lattice vectors
nat= len(mass) #number of atoms in the system
stress= atoms.get_stress()
strten_in= get_sigma(stress)

In [6]:
#this part is equivalent to ac.get_xred_fcart
#in the mdwc_ script
r_t= atoms.get_positions().T
x_t= np.dot(np.linalg.inv(h_t),r_t) # reduced coordinates
f_t= atoms.get_forces().T

In [7]:
#starting velocities
temp= temp_arra[0]
v_t= md_ft.npt_md_suit.init_vel_atoms(mass, temp, len(mass)) #atoms velociites
h_t_dot= md_ft.npt_md_suit.init_vel_lattice(bmass, temp, h_t)
x_t_dot= md_ft.npt_md_suit.get_x_dot(h_t, v_t, nat)

In [8]:
#start of the molecular dynamics
for i_abinit_step in range(abinit_steps):
#abinit_steps is the number of steps in which the forces and energy are
#going to be calculated.
    
    temp= temp_arra[i_abinit_step*md_steps]
    
    s_out, s_out_dot, x_out, v_out= md_ft.npt_md_suit.md_nvt_constrains(h_t,x_t, \
                                                        f_t,v_t,\
                                                        bond_valu, angl_valu,\
                                                        atom_fix_valu,atom_fix_cord,mass,\
                                                        Qmass, dt, temp, s_t, s_t_dot,\
                                                        bond_const,angl_const,\
                                                        atom_fix_const,correc_steps, md_steps,\
                                                        bool_bond_cons,bool_angl_cons,\
                                                        bool_atom_fix_cons, nat,\
                                                        numb_angl_cons, numb_bond_cons,numb_atom_fix_cons)
    x_t= x_out
    v_t= v_out
    s_t= s_out
    s_t_dot=s_out_dot    
    
    #update the information for ase energy and forces calculations
    r_t= np.dot(h_t,x_t)
    atoms.set_positions(r_t.T)
    f_t= atoms.get_forces().T